<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/Copy_of_demo_insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaParse - Fast checking Insurance Contract for Coverage

<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/demo_insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will look at how LlamaParse can be used to extract structured coverage information from an insurance policy.

## Installation of required packages

In [1]:
!pip install llama-index llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


## Download an insurance policy fron IRDAI

The Insurance Regulatory and Development Authority of India (IRDAI) maintains a great resource: https://policyholder.gov.in/web/guest/non-life-insurance-products where all insurance policies available in India are publicly available for download! Let's download a complex health insurance policy as an example.

In [2]:
!wget "https://policyholder.gov.in/documents/37343/931203/NBHTGBP22011V012223.pdf/c392bcc1-f6a8-cadd-ab84-495b3273d2c3?version=1.0&t=1669350459879&download=true" -O "./policy.pdf"

--2024-08-02 15:24:05--  https://policyholder.gov.in/documents/37343/931203/NBHTGBP22011V012223.pdf/c392bcc1-f6a8-cadd-ab84-495b3273d2c3?version=1.0&t=1669350459879&download=true
Resolving policyholder.gov.in (policyholder.gov.in)... 13.107.246.67
Connecting to policyholder.gov.in (policyholder.gov.in)|13.107.246.67|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1341586 (1.3M) [application/pdf]
Saving to: ‘./policy.pdf’

./policy.pdf        100%[===================>]   1.28M  1.20MB/s    in 1.1s    

2024-08-02 15:24:07 (1.20 MB/s) - ‘./policy.pdf’ saved [1341586/1341586]



## Initializing LlamaIndex and LlamaParse

In [3]:
# llama-parse is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
import os
from google.colab import userdata
os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('llama')
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [7]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

# for the purpose of this example, we will use the small model embedding and gpt3.5
embed_model=OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-3.5-turbo-0125")

Settings.llm = llm

## Vanilla Approach - Parse the Policy with LlamaParse into Markdown

In [8]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data("./policy.pdf")

Started parsing the file under job_id bcfbb236-e10a-4210-a2c9-9d473e2a83d0


In [17]:
print(documents[0].text[0:1000])

# Preamble

This ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in advance and the terms, conditions and exclusions of this Policy. Expense incurred outside the policy period will NOT be covered. Unutilized Sum Insured will expire at the end of policy year. All applicable benefits, details and limits are mentioned in your Certificate of insurance. We will cover only allopathic treatments in this policy.

niva Bupa |Healun inauronce

# Defined Terms

The terms listed below in this Section and used elsewhere in the Policy in Initial Capitals shall have the meaning set out against them in this Section.

# Standard Definitions

|2.1|Accident or Accidental|means sudden, unforeseen and involuntary event caused by external, visible and violent means.|
|---|---|---|
|2.2|Co-payment|means a cost sharing requirement under a health insurance policy that provides that the policyholder/insured will bear a specified percentage of th

### Markdown Element Node Parser
Our markdown element node parser works well for parsing the markdown output of LlamaParse into a set of table and text nodes.

In [9]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)

In [10]:
nodes = node_parser.get_nodes_from_documents(documents)

3it [00:00, 19036.18it/s]
100%|██████████| 3/3 [00:02<00:00,  1.06it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 8811.56it/s]
100%|██████████| 1/1 [00:01<00:00,  1.67s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 8305.55it/s]
100%|██████████| 1/1 [00:01<00:00,  1.27s/it]
1it [00:00, 1637.76it/s]
100%|██████████| 1/1 [00:06<00:00,  6.66s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
2it [00:00, 3160.74it/s]
100%|██████████| 2/2 [00:02<00:00,  1.08s/it]
1it [00:00, 8774.69it/s]
100%|██████████| 1/1 [00:01<00:00,  1.72s/it]
1it [00:00, 7169.75it/s]
100%|██████████| 1/1 [00:01<00:00,  1.43s/it]
3it [00:00, 24105.20it/s]
100%|██████████| 3/3 [00:06<00:00,  2.25s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
4it [00:00, 33156.55it/s]
100%|██████████| 4/4 [00:02<00:00,  1.53it/s]
5it [00:00, 11360.52it/s]
100%|██████████| 5/5 [00:43<00:00,  8.61s/it]
0it [00:00, ?it/s]
0

In [11]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [12]:
query_engine = recursive_index.as_query_engine(similarity_top_k=25)

### Querying the model for coverage

In [14]:
query_1 = "My trip was delayed and I paid 45, how much am I covered for?"

response_1 = query_engine.query(query_1)
print(str(response_1))

You are covered for the fixed amount as mentioned in the certificate of insurance for every block of hours of delay, up to a maximum of 24 hours of delay.


In [13]:
query_1 = "is my dental checkup covered?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Emergency dental treatment is covered under the insurance policy, with coverage ranging from USD 200 to USD 1000 in multiples of 100.


The information is split across the document which leads to retrieval issues. Let's try some parsing instructions to improve our result.

In [17]:
documents_with_instruction = LlamaParse(result_type="markdown", parsing_instruction="""
This document is an insurance policy.
When a benefit/coverage/exlusion is described in the document append a line of the following benefits string format (where coverage could be an exclusion).

For {nameofrisk} and in this condition {whenDoesThecoverageApply} the coverage is {coverageDescription}.

If the document contains a benefits TABLE that describe coverage amounts, do not ouput it as a table, but instead as a list of benefits strings.

""").load_data("./policy.pdf")

Started parsing the file under job_id 52131011-77ba-4be5-a39d-8ca4fc58b72d


Let see how the 2 parsing compare (change target page to explore)

In [16]:
target_page = 45
pages_vanilla = documents[0].text.split("\n---\n")
pages_with_instructions = documents_with_instruction[0].text.split("\n---\n")

print(pages_vanilla[target_page])
print("\n\n=========================================================\n\n")
print(pages_with_instructions[target_page])

IndexError: list index out of range

In [18]:
node_parser_instruction = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)
nodes_instruction = node_parser.get_nodes_from_documents(documents_with_instruction)
base_nodes_instruction, objects_instruction = node_parser_instruction.get_nodes_and_objects(nodes_instruction)

recursive_index_instruction = VectorStoreIndex(nodes=base_nodes_instruction+objects_instruction)
query_engine_instruction = recursive_index_instruction.as_query_engine(similarity_top_k=25)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, 

## Comparing Instruction-Augmented Parsing vs. Vanilla Parsing

When we parse the document with natural language instructions to add context on insurance coverage, we are able to correctly answer a wide range of queries in our RAG pipeline. In contrast, a RAG pipeline built with the vanilla method is not able to answer these queries.

In [19]:
query_1 = "My trip was delayed and I paid 45, how much am I covered for?"

response_1 = query_engine.query(query_1)
print("Vanilla:")
print(response_1)

print("With instructions:")
response_1_i = query_engine_instruction.query(query_1)
print(response_1_i)


Vanilla:
You are covered for the fixed amount mentioned in the certificate of insurance for every block of hours of delay, up to a maximum of 24 hours of delay.
With instructions:
You are covered for the delay of your trip up to the amount specified in the Certificate of Insurance.


Looking at the policy it says in list I that one expense not covered is Baby food

In [20]:
query_2 = "I just had a baby, is baby food covered?"

response_2 = query_engine.query(query_2)
print("Vanilla:")
print(response_2)

print("With instructions:")
response_2_i = query_engine_instruction.query(query_2)
print(response_2_i)

Vanilla:
Baby food expenses are not covered under the policy.
With instructions:
Baby food is not covered under the insurance policy.


In [ ]:
query_3 = "How is gauze used in my operation covered?"

response_3 = query_engine.query(query_3)
print("Vanilla:")
print(response_3)

print("With instructions:")
response_3_i = query_engine_instruction.query(query_3)
print(response_3_i)

Vanilla:
Gauze used in your operation is covered as part of the surgical procedure carried out in a fully equipped operation theatre of its own, under the supervision of qualified medical practitioners. The insurance company will have access to daily records of patients, including details of procedures like the use of gauze during the operation.
With instructions:
Procedure Charges cover the use of gauze in your operation.
